In [40]:
import numpy as np

In [2]:
%load_ext autoreload
%autoreload 2

## Dataset loading

In [75]:
from utils.dataset_loader import load_datasets

(texts_train, labels_train), (texts_val, labels_val), (texts_test, labels_test) = load_datasets(folder_path='dataset', 
                                                                                                divide_by_sentence=True)


Vectorizing the texts

In [76]:
from tensorflow.keras.layers import TextVectorization

texts = texts_train + texts_val + texts_test

sequence_length = 50

vectorizer = TextVectorization(output_sequence_length=sequence_length, standardize=None)  # standaridze=None

vectorizer.adapt(data=texts)

x = vectorizer(texts)

# Mapping from integers to word types
vocabulary = np.array(vectorizer.get_vocabulary())

x_train = x[:len(texts_train)]
x_val = x[len(texts_train):len(texts_train)+len(texts_val)]
x_test = x[len(texts_train)+len(texts_val):]

Vectorizing the labels

In [81]:
from tensorflow.keras.layers import TextVectorization

labels = labels_train + labels_val + labels_test

sequence_length = 50

vectorizer_labels = TextVectorization(output_sequence_length=sequence_length, standardize=None)  # standaridze=None

vectorizer_labels.adapt(data=labels)

y = vectorizer_labels(labels)

# Mapping from integers to POS tags
vocabulary_labels = np.array(vectorizer_labels.get_vocabulary())

y_train = y[:len(labels_train)]
y_val = y[len(labels_train):len(labels_train)+len(labels_val)]
y_test = y[len(labels_train)+len(labels_val):]

n_classes = len(vocabulary_labels)

Glove embeddings

In [82]:
from utils.glove_loader import load_glove

embedding_dimension = 50

GLOVE_embeddings = load_glove(folder_path='glove_pretrained', embedding_dim=embedding_dimension)

Embedding matrix

In [83]:
from utils.embedding_matrix_builder import build_embedding_matrix

embedding_matrix = build_embedding_matrix(vocabulary=vocabulary, GLOVE_embeddings=GLOVE_embeddings, embedding_dimension=embedding_dimension)

## Baseline Model

In [84]:
from models.baseline_model import build_baseline_model

baseline_model = build_baseline_model(sequence_length=sequence_length, n_classes=n_classes, embedding_matrix=embedding_matrix)

In [78]:
baseline_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 50)]              0         
                                                                 
 embedding_2 (Embedding)     (None, 50, 50)            547450    
                                                                 
 lstm_2 (LSTM)               [(None, 50, 128),         91648     
                              (None, 128),                       
                              (None, 128)]                       
                                                                 
 dense_1 (Dense)             (None, 50, 47)            6063      
                                                                 
Total params: 645,161
Trainable params: 645,161
Non-trainable params: 0
_________________________________________________________________


In [85]:
import tensorflow as tf

baseline_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])

In [26]:
# Without using punctuation
baseline_model.fit(x=x_train, y=y_train, batch_size=32, epochs=50, validation_data=(x_val, y_val))

Epoch 1/50
62/62 [==============================] - 11s 95ms/step - loss: 2.6905 - accuracy: 0.2862 - val_loss: 2.1904 - val_accuracy: 0.4313
Epoch 2/50
62/62 [==============================] - 3s 55ms/step - loss: 1.7760 - accuracy: 0.5347 - val_loss: 1.5030 - val_accuracy: 0.6178
Epoch 3/50
62/62 [==============================] - 3s 55ms/step - loss: 1.2410 - accuracy: 0.6881 - val_loss: 1.1743 - val_accuracy: 0.7011
Epoch 4/50
62/62 [==============================] - 4s 58ms/step - loss: 0.9665 - accuracy: 0.7597 - val_loss: 1.0111 - val_accuracy: 0.7440
Epoch 5/50
62/62 [==============================] - 4s 57ms/step - loss: 0.8084 - accuracy: 0.8014 - val_loss: 0.9072 - val_accuracy: 0.7682
Epoch 6/50
62/62 [==============================] - 3s 55ms/step - loss: 0.6947 - accuracy: 0.8293 - val_loss: 0.8378 - val_accuracy: 0.7818
Epoch 7/50
62/62 [==============================] - 3s 56ms/step - loss: 0.6081 - accuracy: 0.8478 - val_loss: 0.7827 - val_accuracy: 0.7952
Epoch 8/50
6

In [86]:
baseline_model.fit(x=x_train, y=y_train, batch_size=32, epochs=50, validation_data=(x_val, y_val))

Epoch 1/50
62/62 [==============================] - 12s 94ms/step - loss: 2.8421 - accuracy: 0.2592 - val_loss: 2.2784 - val_accuracy: 0.4334
Epoch 2/50
62/62 [==============================] - 4s 67ms/step - loss: 1.7607 - accuracy: 0.5730 - val_loss: 1.3702 - val_accuracy: 0.6663
Epoch 3/50
62/62 [==============================] - 4s 61ms/step - loss: 1.0657 - accuracy: 0.7450 - val_loss: 0.9284 - val_accuracy: 0.7703
Epoch 4/50
62/62 [==============================] - 4s 62ms/step - loss: 0.7301 - accuracy: 0.8230 - val_loss: 0.7109 - val_accuracy: 0.8193
Epoch 5/50
62/62 [==============================] - 4s 62ms/step - loss: 0.5465 - accuracy: 0.8650 - val_loss: 0.5874 - val_accuracy: 0.8450
Epoch 6/50
62/62 [==============================] - 4s 62ms/step - loss: 0.4328 - accuracy: 0.8914 - val_loss: 0.5110 - val_accuracy: 0.8600
Epoch 7/50
62/62 [==============================] - 4s 62ms/step - loss: 0.3541 - accuracy: 0.9103 - val_loss: 0.4596 - val_accuracy: 0.8716
Epoch 8/50
6

Check the predicted labels on a single test sentence

In [63]:
texts_test[0]

'intelogic trace inc. , san antonio , texas , said it bought 2.7 million shares , or about 18 % , of its common stock from an unaffiliated shareholder for $ 3.625 a share , or $ 9.9 million .'

In [64]:
labels_test[0]

'NNP NNP NNP , NNP NNP , NNP , VBD PRP VBD CD CD NNS , CC IN CD NN , IN PRP$ JJ NN IN DT JJ NN IN $ CD DT NN , CC $ CD CD .'

In [67]:
# Without using punctuation

import numpy as np

vocabulary_labels[np.argmax(baseline_model(x_test[0:1]), axis=2)]

array([['nnp', 'nns', 'nnp', 'nnp', 'nnp', 'nnp', 'vbd', 'prp', 'vbd',
        'cd', 'cd', 'nns', 'cc', 'rb', 'cd', 'nn', 'in', 'prp', 'nn',
        'nn', 'in', 'dt', 'jj', 'nn', 'in', 'jj', 'dt', 'nn', 'cc', 'cd',
        'cd', 'cd', 'cd', 'cd', 'cd', 'cd', 'cd', 'cd', 'cd', 'cd', 'cd',
        'cd', 'cd', 'cd', 'cd', 'cd', 'cd', 'cd', 'cd', 'cd']],
      dtype='<U5')

In [87]:
import numpy as np

vocabulary_labels[np.argmax(baseline_model(x_test[0:1]), axis=2)]

array([['NNP', 'NNS', 'NNP', ',', 'NNP', 'NNP', ',', 'NNP', ',', 'VBD',
        'PRP', 'VBD', 'CD', 'CD', 'NNS', ',', 'CC', 'RB', 'CD', 'NN',
        ',', 'IN', 'PRP$', 'JJ', 'NN', 'IN', 'DT', 'JJ', 'NN', 'IN', '$',
        'CD', 'DT', 'NN', ',', 'CC', '$', 'CD', 'CD', '.', '.', '.', '.',
        '.', '.', '.', '.', '.', '.', '.']], dtype='<U5')

Test score

In [38]:
# Without using puncutation
baseline_model.evaluate(x_test, y_test)

20/20 [==============================] - 1s 23ms/step - loss: 0.9275 - accuracy: 0.8213


[0.9275322556495667, 0.8213129043579102]

In [88]:
baseline_model.evaluate(x_test, y_test)

20/20 [==============================] - 0s 22ms/step - loss: 0.4593 - accuracy: 0.8999


[0.45929422974586487, 0.899909496307373]